In [17]:
import time 
import pandas as pd #pandas: 一個資料分析的函式庫，提供了DataFrame等資料格式，與資料處理的函數。
import numpy as np #numpy: 為科學運算所提供的套件。提供許多數學公式的實作以及高效率的陣列、矩陣運算。
import re #re: 正規表達式套件。
import math #math: 數學運算工具。
import datetime
from pprint import pprint

import seaborn as sns
from matplotlib import font_manager
import matplotlib.pyplot as plt  #matplotlib: 繪圖工具。

import jieba #jieba: 中文斷詞套件。
import jieba.analyse

import pyLDAvis
import pyLDAvis.gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel, CoherenceModel
from gensim.models.ldamulticore import LdaMulticore
from gensim.matutils import corpus2csc, corpus2dense, Sparse2Corpus

import guidedlda

from wordcloud import WordCloud 
from nltk import ngrams, FreqDist #NLTK: 全名為Natural Language Tool Kit，自然語言處理工具。
from collections import Counter, namedtuple #collections: 提供額外的數據類型。
import networkx as nx #networkx: 網絡圖建構、繪製工具。
from snownlp import SnowNLP #SnowNLP 針對中文設計，可以提取句子的情緒分類
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER

from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer #CountVectorizer, TfidfTransformer: sklearn中計算詞頻與tf-idf的套件。
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict, KFold
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    roc_curve,
    auc,
    precision_recall_curve,
    RocCurveDisplay
)

In [18]:
#匯入資料
df = pd.read_csv('./data/PTT_Tesla_2023.csv', encoding = 'UTF-8')
df.dropna(subset = ['artContent'], axis=0, how='any', inplace=True)
# df = df[['artDate','artTitle','artContent']]
df['artDate'] = pd.to_datetime(df['artDate'])
MetaData = df.copy()

# 去除一些不需要的欄位
MetaData = MetaData.drop(['artPoster', 'artCatagory', 'artComment', 'e_ip', 'insertedDate', 'dataSource'], axis=1)
# 只留下中文字
MetaData['sentence'] = MetaData['artContent'].apply(lambda x: re.sub('[^\u4e00-\u9fff]+', '',x))

raw_data = MetaData.copy()
raw_data['content'] = raw_data['sentence']
raw_data = raw_data.loc[:,["artDate", "content", ]]  # 文章內容 文章連結
raw_data.head(3)

missing_values = raw_data[raw_data["content"].isnull()]
print("具有缺失值的行：")
print(missing_values)

具有缺失值的行：
Empty DataFrame
Columns: [artDate, content]
Index: []


In [19]:
# 設定繁體中文詞庫
jieba.set_dictionary('./dict/dict.txt.big')
jieba.load_userdict('dict/user_dict.txt')
# 新增stopwords
with open('./dict/stop_words.txt',encoding="utf-8") as f:
    stopWords = [line.strip() for line in f.readlines()]

# 設定斷詞 function
def getToken(row):
    seg_list = jieba.lcut(row)
    seg_list = [w for w in seg_list if w not in stopWords and len(w)>1] # 篩選掉停用字與字元數小於1的詞彙
    return seg_list

Building prefix dict from /Users/tsaipingyu/Library/CloudStorage/GoogleDrive-jerryone888@gmail.com/其他電腦/我的筆記型電腦/中山課程_研究所/碩一下/社媒/SMA_project/蔡秉祐/dict/dict.txt.big ...
Loading model from cache /var/folders/4p/hs58l6r50xd2hy79hrrr3df80000gn/T/jieba.u2b6727b87230940dce34780cfea72435.cache
Loading model cost 0.400 seconds.
Prefix dict has been built successfully.


In [40]:
raw_data['word'] = raw_data.content.apply(getToken)
raw_data

,artDate,content,word
0,2023-01-01 10:48:19,八卦是官網特斯拉二手車比很多第三方平台都便宜等等黨會勝利嗎,"[八卦, 官網, 特斯拉, 二手車, 第三方, 平台, 便宜, 黨會, 勝利]"
1,2023-01-01 22:42:25,各位前輩好想買中古特斯拉里程約萬上下的擔心電池久了會掉電越充越少電里程數越少開了年後車子變成...,"[前輩, 中古, 特斯拉, 里程, 擔心, 電池, 掉電, 越充, 越少電, 里程, 數越,..."
2,2023-01-02 12:47:31,原文連結原文內容記者王家珩徐兆緯台北報導前電競世界冠軍日前開直播雖然用自動駕駛但雙手卻完全離...,"[王家, 徐兆緯, 台北, 前電, 世界冠軍, 日前, 直播, 自動, 駕駛, 雙手, 完全..."
3,2023-01-02 18:01:04,如題本身開特斯拉最近出門覺得找車位很煩都騎家裡超舊機車基本上在路上有機車道騎機車道沒機車道就...,"[特斯拉, 最近, 出門, 覺得, 車位, 很煩, 騎家裡, 超舊, 機車, 路上, 有機,..."
4,2023-01-02 19:21:24,原文連結原文內容美國汽車召回排名揭曉第一名超過萬輛車進廠處理結束迎來許多年的數據也立刻揭曉來...,"[美國, 汽車, 召回, 揭曉, 第一名, 超過, 進廠, 處理, 結束, 迎來, 許多年,..."
...,...,...,...
1218,2023-12-30 13:22:05,特斯拉超充要花費的成本是最低的因為土地地主要無償提供給特斯拉電車網紅有訪問過超充地主你說連地...,"[特斯拉, 超充, 要花費, 成本, 最低, 土地, 主要, 無償, 提供, 特斯拉, 電車..."
1219,2023-12-30 16:47:46,最近特斯拉新更新把輔助駕駛搭配車內鏡頭變得更嚴格總之會偵測你眼睛有沒有在看前方或者是不是在睡...,"[最近, 特斯拉, 更新, 輔助, 駕駛, 搭配, 鏡頭, 變得, 嚴格, 偵測, 眼睛, ..."
1220,2023-12-30 17:44:17,好有共鳴雖然我不是小眼睛但我是斜視看前面就像吉娃娃的眼睛也是一直被警告後來我把內鏡頭貼起來就...,"[共鳴, 眼睛, 斜視, 前面, 吉娃娃, 眼睛, 一直, 警告, 鏡頭, 沒事, 車內, 鏡頭]"
1221,2023-12-30 21:39:18,心得特斯拉也太不衛生了吧這是最低兩百萬最高到四五百萬的豪車耶怎可以有床蝨這種髒東西還整船被澳...,"[特斯拉, 衛生, 這是, 最低, 兩百萬, 最高, 四五百萬, 豪車, 床蝨, 髒東西, ..."


In [53]:
word_list=raw_data['word'].explode('word').tolist()
len(word_list)

165471

In [54]:
word_unique_list=list(set(word_list))
word_unique_list = [x for x in word_unique_list if not (isinstance(x, float) and math.isnan(x))]
word_unique_list

['薩克森',
 '說試',
 '起跑點',
 '錯過',
 '假皮',
 '減少',
 '主牌',
 '利益',
 '俄連',
 '基建',
 '連假',
 '車馬',
 '哀配',
 '並使',
 '保持一致',
 '代表',
 '計入',
 '誇獎',
 '公司員工',
 '破裂',
 '緊急狀況',
 '三五年',
 '那油',
 '排充',
 '軍職',
 '演變成',
 '太苦',
 '方法',
 '禁止',
 '找出',
 '訂立',
 '吸引力',
 '首年',
 '財閥',
 '財政部',
 '改判',
 '科學課',
 '不盤',
 '場六高雄場',
 '組內',
 '到紐',
 '萬能',
 '老氣',
 '油壓',
 '二期',
 '得易科',
 '友人',
 '大開',
 '車撞',
 '緩不濟急',
 '令人不安',
 '站驗車',
 '順延',
 '撞毀',
 '讓後車',
 '震懾',
 '瑞光',
 '高代',
 '萬零',
 '灌單',
 '某種意義',
 '澤拉雅',
 '時有發生',
 '違反',
 '更快',
 '到位',
 '兩噸',
 '三星',
 '下車',
 '美金',
 '財力雄厚',
 '懶得',
 '神奇',
 '反智',
 '六八',
 '省下光',
 '大笑話',
 '多種',
 '禮貌',
 '綁乙式',
 '整理出來',
 '擱置',
 '換檔',
 '多年',
 '高速成長',
 '焦慮',
 '地方',
 '副理',
 '探歌',
 '冷卻水',
 '這倒',
 '凌志',
 '買個',
 '雖穩',
 '不好玩',
 '環環相扣',
 '車數少',
 '不景氣',
 '義務教育',
 '丟到',
 '嚇了',
 '競爭者',
 '零首付',
 '手超',
 '拼好',
 '解釋一下',
 '預保',
 '預處理',
 '並將',
 '台車',
 '降價',
 '太近',
 '對手',
 '了當',
 '續航',
 '著喊',
 '邊看邊',
 '發覺',
 '偷輕',
 '矯正',
 '賣方',
 '降維',
 '割喉',
 '夫妻',
 '破相',
 '最少',
 '廢料',
 '那是還',
 '有裝',
 '沒開',
 '飛行員',
 '多款',
 '加價',
 '萬含',
 '防疫',
 '

In [56]:
# 假设 word_unique_list 是你的列表
if "特斯拉" in word_unique_list:
    print("word_unique_list 中包含 '特斯拉'")
else:
    print("word_unique_list 中不包含 '特斯拉'")

word_unique_list 中包含 '特斯拉'


In [37]:
word_list=list()
word_list

[nan,
 nan,
 nan,
 nan,
 '薩克森',
 '說試',
 '起跑點',
 '錯過',
 '假皮',
 '減少',
 '主牌',
 '利益',
 '俄連',
 '基建',
 '連假',
 '車馬',
 '哀配',
 '並使',
 '保持一致',
 '代表',
 '計入',
 '誇獎',
 '公司員工',
 '破裂',
 '緊急狀況',
 '三五年',
 '那油',
 '排充',
 '軍職',
 '演變成',
 '太苦',
 '方法',
 '禁止',
 '找出',
 '訂立',
 '吸引力',
 '首年',
 '財閥',
 '財政部',
 '改判',
 '科學課',
 '不盤',
 '場六高雄場',
 '組內',
 '到紐',
 '萬能',
 '老氣',
 '油壓',
 '二期',
 '得易科',
 '友人',
 '大開',
 '車撞',
 '緩不濟急',
 '令人不安',
 '站驗車',
 '順延',
 '撞毀',
 '讓後車',
 '震懾',
 '瑞光',
 '高代',
 '萬零',
 '灌單',
 '某種意義',
 '澤拉雅',
 '時有發生',
 '違反',
 '更快',
 '到位',
 '兩噸',
 '三星',
 '下車',
 '美金',
 '財力雄厚',
 '懶得',
 '神奇',
 '反智',
 '六八',
 '省下光',
 '大笑話',
 '多種',
 '禮貌',
 '綁乙式',
 '整理出來',
 '擱置',
 '換檔',
 '多年',
 '高速成長',
 '焦慮',
 '地方',
 '副理',
 '探歌',
 '冷卻水',
 '這倒',
 '凌志',
 '買個',
 '雖穩',
 '不好玩',
 '環環相扣',
 '車數少',
 '不景氣',
 '義務教育',
 '丟到',
 '嚇了',
 '競爭者',
 '零首付',
 '手超',
 '拼好',
 '解釋一下',
 '預保',
 '預處理',
 '並將',
 '台車',
 '降價',
 '太近',
 '對手',
 '了當',
 '續航',
 '著喊',
 '邊看邊',
 '發覺',
 '偷輕',
 '矯正',
 '賣方',
 '降維',
 '割喉',
 '夫妻',
 '破相',
 '最少',
 '廢料',
 '那是還',
 '有裝',
 '沒開',
 '飛行員',
 '多款',

In [9]:
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY")
)

def get_embedding_openai(text, model):
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [10]:
# 回傳cosine similarity分數
def get_result_df(sentences, cosine_scores):

  result = []
  for i in range(len(sentences)):
      for j in range(i+1, len(sentences)):
          result.append([sentences[i], sentences[j], cosine_scores[i][j].item()])

  result_df = pd.DataFrame(result, columns=["sentence1", "sentence2", "score"])
  result_df = result_df.sort_values("score", ascending = False)

  return result_df

In [11]:
embedding_model = "text-embedding-3-small"
embedding_encoding = "cl100k_base"
max_tokens = 8000  # the maximum for text-embedding-3-small is 8191

In [15]:
# 示範句子
sentences = ['八卦', '官網', '特斯拉', '二手車', '第三方', '平台', '便宜', '黨會', '勝利']

# 取得每一句的embedding
embeddings_openai = [get_embedding_openai(text, model=embedding_model) for text in sentences]

# Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings_openai, embeddings_openai)

# 印出句子間的cosine similarity分數
result_df = get_result_df(sentences, cosine_scores)
result_df

,sentence1,sentence2,score
23,二手車,便宜,0.437549
26,第三方,平台,0.386583
11,官網,平台,0.369942
10,官網,第三方,0.339385
34,便宜,勝利,0.321364
21,二手車,第三方,0.319433
22,二手車,平台,0.305908
30,平台,便宜,0.298214
12,官網,便宜,0.298207
13,官網,黨會,0.297934


In [16]:
def get_dimensions(lst):
    if isinstance(lst, list):
        return [len(lst)] + get_dimensions(lst[0]) if len(lst) > 0 else []
    else:
        return []
get_dimensions(embeddings_openai)
# 每個字都有1536維度

[9, 1536]